# Machine Learning Pipeline with KubeDirector - Lab 2
## Deploy a local Jupyter Notebook cluster to interact with a tenant-shared training cluster

### **Lab workflow**

In this lab:

1. As tenant user, you will first create a local (lightweight) Jupyter Notebook application KubeDirector cluster to develop your model. You will attach your Jupyter Notebook cluster to a remote tenant-shared training cluster to train your model. The shared training cluster **training-engine-shared** includes the open source ML toolkits, libraries and frameworks for developing and training models. It has been already deployed by the tenant administrator for your tenant. The shared training cluster will allow you to train your model faster using more compute and memory resources than your local lightweight Jupyter Notebook cluster.

2. You will then access your local Jupyter Notebook web UI via the gateway network service port to train your model to the remote tenant-shared training cluster. 

**Recommended blog reading:**

* [Building Dynamic Machine Learning Pipelines with KubeDirector](https://developer.hpe.com/blog/building-dynamic-machine-learning-pipelines-with-kubedirector)

**Definitions:**

- *KubeDirector:* also known as Kubernetes Director. KubeDirector is an **open-source** project initiated and led by HPE that addresses stateful scaleout application deployment in standard Kubernetes clusters with a focus on non-cloud native stateful analytics workloads (AI/ML, data processing and Big Data apps). These applications are generally referred to as a distributed, single-node or multi-node application **virtual cluster** where each application virtual cluster node runs as **a container** in the Kubernetes cluster.

- *Training:* Input datasets are processed to create a Machine Learning model. Data scientists can use a local Jupyter Notebook to build and train their models. They can also interact with a remote, larger capacity training cluster to train their models faster.

- *Cloud native application:* Also known as the [12-Factor app](https://www.mirantis.com/blog/how-do-you-build-12-factor-apps-using-kubernetes/), a modern application that leverages a microservices architecture with loosely coupled services. The microservice architectural style is an approach to developing a single application as a suite of small independently deployed services.

- *Non-cloud native application:* a multi-tier application with tightly coupled and interdependent services. 

- *Stateless application:* A stateless application is an application which does not require persistence of data nor an application state.

- *Stateful application:* A stateful application typically requires persistence of certain mountpoints across application cluster nodes rescheduling, restarts, upgrades, rollbacks. A stateful application can also need persistence of network identity (i.e.: hostname). 

### **1- Initialize the environment**

Let's first define the environment variables needed to execute this part of the lab.

In [1]:
#
# environment variables
#
studentId="student75" # your Jupyter Notebook student Identifier (i.e.: student<xx>)

#
gateway_host="haecpgtw.etc.fr.comm.hpecorp.net"
Internet_access="notebooks.hpedev.io"

JupyterNotebookApp="cr-cluster-jupyter-notebook.yaml" # the Jupyter Notebook KD App manifest you will deploy to build your model
DeploymentEngineApp="cr-cluster-endpoint-wrapper.yaml" # the Deployment engine KD App manifest you will deploy to query your model for answers

echo "Your studentId is: "$studentId

Your studentId is: student75


### **2- List the registered KubeDirector applications**
You can get the list of KubeDirector applications (kdapp) registered with the Kubernetes cluster for your tenant using the `kubectl get kdapp` command. A KubeDirector application is a _template_ for the application. It describes an application's **metadata** (service roles, Docker images, configuration packages, services ports, persistent storage).
In this workshop, you will be using the KubeDirector application _jupyter-notebook-v1_ to create your local Jupyter Notebook cluster.

In [2]:
kubectl get kdapp

NAME                     AGE
centos8x                 26d
deployment-engine        26d
jupyter-notebook         26d
jupyter-notebook-v1      26d
spark245                 26d
tensorflow-gpu-jupyter   26d
training-engine          26d
ubuntu18x                26d


List the instance of the training engine kdapp already deployed for your tenant by the lab administrator:

In [3]:
kubectl get kdcluster training-engine-shared

NAME                     AGE
training-engine-shared   23d


### **3- Deploying your local Jupyter Notebook cluster with _Connection_ to a remote tenant-shared training cluster**

You will deploy an instance of the **jupyter-notebook-v1** kdapp by creating a KubeDirector virtual cluster (kdcluster). A kdcluster identifies the desired kdapp and specifies runtime configuration parameters, such as the size and resource requirements of the virtual cluster. 

> **Note:** _The Jupyter Notebook kdapp includes the open source machine learning toolkits, software libraries and frameworks for developing and training models such as TensorFlow, scikit-learn, keras, XGBoost, matplotlib, Jupyter Notebook, Numpy, Scipy, Pandas, etc._

#### Create the manifest file and deploy an instance of the Jupyter Notebook KubeDirector application:
Like any other containerized application deployment on Kubernetes, the `kubectl apply -f ManifestAppFile` command is used to deploy the kdcluster. The application manifest is a YAML file that describes the attributes of the KubeDirector virtual cluster.  

> **Note:** _One of the most interesting parts of the kdcluster specification is the **Connections** stanza (a related group of attributes), which identifies other resources of interest to that kdcluster. Here, you simply connect your local Jupyter Notebook cluster to the tenant-shared training cluster **training-engine-shared** already deployed by the tenant administrator._

In [4]:
cat $JupyterNotebookApp

apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata:
  name: "jupyter-notebook-student75"
spec:
  app: "jupyter-notebook-v1"
  appCatalog: "local"
  connections: 
    #secrets: 
      #- 
        #"some secrets"
    #configmaps: 
      #- 
        #"some configmaps"
    clusters: 
      - "training-engine-shared"
        #"some clusters"
  roles:
  - id: controller
    resources:
      requests:
        memory: "2Gi"
        cpu: "1"
      limits:
        memory: "2Gi"
        cpu: "1"


In [5]:
kubectl apply -f $JupyterNotebookApp

kubedirectorcluster.kubedirector.hpe.com/jupyter-notebook-student75 created


After a few seconds, you should get the response message: *kubedirectorcluster/Your-instance-name created*.  

### **4- Inspect the deployed KubeDirector Application instance**
Your application will be represented in the Kubernetes cluster by a custom resource of type **KubeDirectorCluster (kdcluster)**, with the name that was indicated inside the YAML file used to create it. Use the command `kubectl get kdcluster YourClustername` to list your kdcluster.

In [6]:
clusterName="jupyter-notebook-${studentId}"
kubectl get kdcluster $clusterName

NAME                         AGE
jupyter-notebook-student75   5s


After creating the instance of the KubeDirector application, you can use the `kubectl describe kdcluster` command below to observe its status and any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

> **Note:** _The first time a virtual cluster of a given KubeDirector application type is created, it may take several minutes to reach its **"configured"** state, as the relevant Docker image must be downloaded and imported._ 

**>Run the `kubectl describe` command below and scroll down to the `Events` section to check the overall state of your kdcluster.**

**>Regularly repeat (every minute or so) the command below until the kdcluster is in the state "_configured_".**

In [8]:
kubectl describe kdcluster $clusterName

Name:         jupyter-notebook-student75
Namespace:    k8smltenant
Labels:       <none>
Annotations:  <none>
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorCluster
Metadata:
  Creation Timestamp:  2021-01-14T07:42:29Z
  Finalizers:
    kubedirector.hpe.com/cleanup
  Generation:        1
  Resource Version:  9242541
  Self Link:         /apis/kubedirector.hpe.com/v1beta1/namespaces/k8smltenant/kubedirectorclusters/jupyter-notebook-student75
  UID:               e43e3739-09d9-4ad8-80c0-232243a9f208
Spec:
  App:          jupyter-notebook-v1
  App Catalog:  local
  Connections:
    Clusters:
      training-engine-shared
  Naming Scheme:  UID
  Roles:
    Id:       controller
    Members:  1
    Resources:
      Limits:
        Cpu:     1
        Memory:  2Gi
      Requests:
        Cpu:     1
        Memory:  2Gi
  Service Type:  NodePort
Status:
  Cluster Service:       kdhs-qrjqc
  Generation UID:        070478e0-9c89-4c70-8310-56861cdc2748
  Last Connection Hash:  

You can use a form of the `kubectl get all` command that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label to observe the standard Kubernetes resources that compose the application virtual cluster:

In [9]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

NAME               READY   STATUS    RESTARTS   AGE
pod/kdss-dzkdh-0   1/1     Running   0          84s

NAME                     TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                       AGE
service/kdhs-qrjqc       ClusterIP   None            <none>        8888/TCP                      84s
service/s-kdss-dzkdh-0   NodePort    10.110.58.105   <none>        22:30618/TCP,8888:30028/TCP   84s

NAME                          READY   AGE
statefulset.apps/kdss-dzkdh   1/1     84s


Your instance of the KubeDirector Application virtual cluster is made up of a **StatefulSet**, a **POD** (a cluster node) and a **NodePort Service** per service role member (Controller), and a **headless service** for the application cluster.   

* The ClusterIP service is the headless service required by a Kubernetes StatefulSet to work. It maintains a stable POD network identity (i.e.: persistence of the hostname of the PODs across PODs rescheduling).
* The NodePort service exposes the Notebook application service with token-based authorization outside the Kubernetes cluster. 

### **5- Get your local Jupyter Notebook's service endpoint and the authentication password to connect to it**
To get a report on all services related to a specific virtual KubeDirector cluster, you can use a form of **kubectl describe** that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [10]:
#
# Getting the service endpoint URL:
#
JupyterAppURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway/8888 | awk '{print $2}')
JupyterAppPort=$(echo $JupyterAppURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myJupyterApp_endpoint="https://$gateway_host:$JupyterAppPort"
echo "Your application service endpoint re-mapped port is: "$JupyterAppPort
#echo "Your Intranet application service endpoint is: "$myJupyterApp_endpoint
echo "Your Jupyter Notebook service endpoint URL is: https://"$Internet_access:$JupyterAppPort
#
# Getting the auth-token:
#
JupyterAuthToken=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep kd-auth-token | awk '{print $2}')
echo "Your Jupyter Notebook service authentication password is: "$JupyterAuthToken

Your application service endpoint re-mapped port is: 10047
Your Jupyter Notebook service endpoint URL is: https://notebooks.hpedev.io:10047
Your Jupyter Notebook service authentication password is: 253fa255428af14a7b4b1075c93760c4


### **6- Download the python code files to your PC/laptop**

Download the files below on your local PC/laptop. You will use these files in the next part of the lab from your local Jupyter Notebook cluster you have just created.

- _**3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb**_
- _**XGB_Scoring.py**_
- _**XGB_Scoringv2.py**_
- _**ML-Workflow.jpg**_

From the left side panel of your JupyterHub account, navigate to **Code/NYCTaxi** folder: double-click on the folder **Code**, then the folder **NYCTaxi**. Right-click on each file (or select all the files then right-click) and choose **Download**.

You can click the ellipsis **(...)** to go back to the root of your repository in JupyterHub.

## **7- Connect to your local Jupyter Notebook web UI and upload code files**
Click the service endpoint URL from Step 5 above to connect to your Jupyter Notebook server.
This opens a Jupyter Notebook login screen in a new browser tab. Login with the authentication password above (copy/paste the authentication password value above) and follow instructions below to upload the code files from your local PC/laptop to your local Jupyter Notebook server.

![Jupyter-Notebook-Login](Pictures/Jupyter-Notebook-Login.png)

Click on **Upload** button on the top right of your local Jupyter Notebook server.

![Jupyter-Notebook-Upload](Pictures/Jupyter-Notebook-Upload.png)

Select the Lab 3 python code file **3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb** from your laptop downloaded at the beginning of this lab, and click on **Upload** button as shown here:

![Lab3 code upload](Pictures/Jupyter-Notebook-Upload-file.png)

>**Note:** <font color="red">The upload operation may get stuck. If this is the case, refresh your browser tab (CTRL/F5) and repeat the upload operation.</font>

Repeat the same steps to upload the files **XGB_Scoring.py**, **XGB_Scoringv2.py** and **ML-Workflow.jpg** from your laptop downloaded at the beginning of this lab.

### <font color="red">Now, from your local Jupyter Notebook, open the notebook **3-WKSHP-K8s-ML-Pipeline-Model-Training.ipynb** and follow the instructions from the notebook to build, train and test the model.</font>

Once your model is trained and saved to a file, follow the instructions in Lab 4 to deploy your trained model:

* [Lab 4 Model Registry and Deployment](4-WKSHP-K8s-ML-Pipeline-Register-Model-Deployment.ipynb)

## Summary

In this lab, we have shown you how you can deploy a local Jupyter Notebook virtual cluster and attach it to a shared distributed training cluster using _**Connections**_ stanza in a KubeDirector Cluster manifest YAML file. This local Jupyter Notebook will be used in the next lab to do model training on the tenant-shared training cluster.